In [81]:
from __future__ import print_function
from __future__ import division

import os
import sys
import time
import datetime
import argparse
import os.path as osp
import numpy as np

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader
from torch.optim import lr_scheduler

from torchreid import data_manager
from torchreid.dataset_loader import ImageDataset
from torchreid import transforms as T
from torchreid import models
from torchreid.losses import CrossEntropyLabelSmooth, DeepSupervision
from torchreid.utils.iotools import save_checkpoint, check_isfile
from torchreid.utils.avgmeter import AverageMeter
from torchreid.utils.logger import Logger
from torchreid.utils.torchtools import set_bn_to_eval, count_num_param
from torchreid.utils.reidtools import visualize_ranked_results
from torchreid.eval_metrics import evaluate
from torchreid.optimizers import init_optim


In [82]:
from IPython.core.debugger import set_trace

In [83]:
dir_path = "data/market1501/"

In [84]:
height, width = 224, 224

In [85]:
dataset = data_manager.init_imgreid_dataset(root='data', name='market1501', split_id='0',
cuhk03_labeled=False, cuhk03_classic_split=False)


=> Market1501 loaded
Dataset statistics:
  ------------------------------
  subset   | # ids | # images
  ------------------------------
  train    |   751 |    12936
  query    |   750 |     3368
  gallery  |   751 |    15913
  ------------------------------
  total    |  1501 |    32217
  ------------------------------


In [86]:
transform_train = T.Compose([
    Random2DTranslation(height, width),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [87]:
transform_test = T.Compose([
    T.Resize((height, width)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [88]:
trainloader = DataLoader(
    ImageDataset(dataset.train, transform=transform_train),
    batch_size = 64, shuffle = True, num_workers = 4
)

In [89]:
queryloader = DataLoader(
    ImageDataset(dataset.query, transform=transform_test),
    batch_size = 64, shuffle = False, num_workers = 4
)

In [90]:
galleryloader = DataLoader(
    ImageDataset(dataset.gallery, transform=transform_test),
    batch_size=64, shuffle=False, num_workers= 4
)

In [91]:
class ResNet50(nn.Module):
    def __init__(self, num_classes, loss={'xent'}, **kwargs):
        super(ResNet50, self).__init__()
        self.loss = loss
        resnet50 = torchvision.models.resnet50(pretraine.Sequential(*list(resnet50.childred=True)
        self.base = nnn())[:-2])
        self.classifier = nn.Linear(2048, num_classes)
        self.feat_dim = 2048

    def forward(self, x):
        x = self.base(x)
        x = F.avg_pool2d(x, x.size()[2:])
        f = x.view(x.size(0), -1)
        if not self.training:
            return f
        y = self.classifier(f)

        if self.loss == {'xent'}:
            return y
        elif self.loss == {'xent', 'htri'}:
            return y, f
        else:
            raise KeyError("Unsupported loss: {}".format(self.loss))


In [115]:
print_freq = 10

In [116]:
def train(epoch, model, criterion, optimizer, trainloader, use_gpu, freeze_bn=False):
    losses = AverageMeter()
    batch_time = AverageMeter()
    data_time = AverageMeter()

    model.train()

    if freeze_bn:
        model.apply(set_bn_to_eval)

    end = time.time()
    for batch_idx, (imgs, pids, _) in enumerate(trainloader):
        data_time.update(time.time() - end)
        
        if use_gpu:
            imgs, pids = imgs.cuda(), pids.cuda()
        
        outputs = model(imgs)
        if isinstance(outputs, tuple):
            loss = DeepSupervision(criterion, outputs, pids)
        else:
            loss = criterion(outputs, pids)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        batch_time.update(time.time() - end)

        losses.update(loss.item(), pids.size(0))

        if (batch_idx + 1) % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.4f} ({data_time.avg:.4f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'.format(
                   epoch + 1, batch_idx + 1, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses))
        
        end = time.time()


In [107]:
criterion = nn.CrossEntropyLoss()

In [118]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [111]:
model = models.init_model(name='resnet50', num_classes=dataset.num_train_pids, loss={'xent'}, use_gpu=True)

In [113]:
model = model.cuda()

In [ ]:
for epoch in range(0,10):
    train(epoch, model, criterion, optimizer, trainloader=trainloader, use_gpu=True)

Epoch: [1][10/203]	Time 0.664 (0.702)	Data 0.0059 (0.0391)	Loss 6.5524 (6.9091)	
Epoch: [1][20/203]	Time 0.664 (0.682)	Data 0.0062 (0.0227)	Loss 6.5005 (6.8297)	
Epoch: [1][30/203]	Time 0.659 (0.675)	Data 0.0062 (0.0172)	Loss 6.7040 (6.7659)	
Epoch: [1][40/203]	Time 0.659 (0.671)	Data 0.0061 (0.0144)	Loss 6.7379 (6.7235)	
Epoch: [1][50/203]	Time 0.662 (0.669)	Data 0.0065 (0.0127)	Loss 6.4561 (6.6886)	
Epoch: [1][60/203]	Time 0.663 (0.667)	Data 0.0065 (0.0116)	Loss 6.3843 (6.6584)	
Epoch: [1][70/203]	Time 0.660 (0.667)	Data 0.0060 (0.0108)	Loss 6.4481 (6.6340)	
Epoch: [1][80/203]	Time 0.664 (0.666)	Data 0.0063 (0.0103)	Loss 6.4512 (6.6099)	
Epoch: [1][90/203]	Time 0.663 (0.666)	Data 0.0058 (0.0098)	Loss 6.3704 (6.5882)	
Epoch: [1][100/203]	Time 0.660 (0.665)	Data 0.0055 (0.0094)	Loss 6.3028 (6.5665)	
Epoch: [1][110/203]	Time 0.660 (0.665)	Data 0.0063 (0.0091)	Loss 6.3785 (6.5441)	
Epoch: [1][120/203]	Time 0.663 (0.665)	Data 0.0061 (0.0089)	Loss 6.1427 (6.5186)	
Epoch: [1][130/203]	Time 